In [ ]:
import pandas as pd
import torch
import tensorflow as tf
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt

# data manuplation




## STEP 1
*Load Your CSV Datase*

In [ ]:
import pandas as pd
df = pd.read_csv("dataset.csv.zip")  # Replace with actual filename
df.head()

,id,file_path,file_size,line_count,extension,language
0,1,Markdown/000001.md,34784,572,md,Markdown
1,2,XML/000002.props,3013,44,props,XML
2,3,Text/000003.txt,1076,21,txt,Text
3,4,Markdown/000004.md,8105,84,md,Markdown
4,5,Markdown/000005.md,2763,41,md,Markdown


In [ ]:
missing = df["code"].isnull().sum()
print(f"Missing code rows: {missing}")

Missing code rows: 0


# STEP 2 : Unzip the Code Files


In [ ]:
import zipfile
with zipfile.ZipFile("dataset.zip", "r") as zip_ref:
    zip_ref.extractall("code_files")  # Creates a folder with all code files

we need to creat a file called code files why i dont know

# STEP 3 : Read Code from File Paths


In [ ]:
import os

def read_code(path):
    full_path = os.path.join("code_files", path)
    try:
        with open(full_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"ERROR: {e}"

df["code"] = df["file_path"].apply(read_code)

# STEP 4 : Save as CSV or JSON


In [ ]:
df.to_csv("structured_cleaned_new_dataset.csv", index=False)

In [ ]:
df.to_json("structured_cleaned_new_dataset.json", orient="records", indent=2)

In [ ]:
from google.colab import files
files.download("structured_cleaned_new_dataset.csv")
files.download("structured_cleaned_new_dataset.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
df = pd.read_csv("structured_cleaned_new_dataset.csv")  # Replace with actual filename
df.head()

,id,file_path,file_size,line_count,extension,language,code,clean_code,clean_line_count,clean_size
0,1,Markdown/000001.md,34784,572,md,Markdown,# Contributing\n\n| Component | Bui...,contributing\n\n component build ...,186,10000
1,2,XML/000002.props,3013,44,props,XML,"﻿<Project ToolsVersion=""15.0"" xmlns=""http://sc...",project toolsversion xmlns\n propertygroup\n ...,44,1812
2,3,Text/000003.txt,1076,21,txt,Text,The MIT License (MIT)\n\nCopyright (c) 2015 Mi...,the mit license mit\n\ncopyright c 2015 micros...,21,1026
3,4,Markdown/000004.md,8105,84,md,Markdown,# Azure SDK for .NET\n\n[![Packages](https://i...,azure sdk for net\n\npackageshttpsimgshieldsi...,84,7244
4,5,Markdown/000005.md,2763,41,md,Markdown,<!-- BEGIN MICROSOFT SECURITY.MD V0.0.5 BLOCK ...,begin microsoft securitymd v005 block \n\n se...,41,2523


# THE CLEAN FUNCTION

the first one clean the codes and make them in only one row

In [ ]:
def preprocess_code(code):
    import re
    if not isinstance(code, str):
        return ""  # or return str(code) if you want to keep it
    code = re.sub(r'\".*?\"|\'.*?\'', '', code)  # remove strings
    code = re.sub(r'\W+', ' ', code)            # remove symbols
    code = re.sub(r'#.*|//.*|/\*.*?\*/', '', code) # remove comment
    code = code[:10000]  # Truncate long files
    return code.lower()

    # here we used it because we get a TypeError: expected string or bytes-like object, got 'float') means that some rows in your "code" column are missing or corrupted, and Python is trying to apply your cleaning function to a NaN or float instead of a string.
# for the preciouse cleanning func
# the func was
'''
def preprocess_code(code):
    import re
    code = re.sub(r'\".*?\"|\'.*?\'', '', code)  # remove strings
    code = re.sub(r'[^a-zA-Z0-9_]', ' ', code)   # remove symbols
    return code.lower()

df["clean_code"] = df["code"].apply(preprocess_code)
'''

'\ndef preprocess_code(code):\n    import re\n    code = re.sub(r\'".*?"|\'.*?\'\', \'\', code)  # remove strings\n    code = re.sub(r\'[^a-zA-Z0-9_]\', \' \', code)   # remove symbols\n    return code.lower()\n\ndf["clean_code"] = df["code"].apply(preprocess_code)\n'

this one let the structer of the code

In [ ]:
def preprocess_code(code):
    import re
    if not isinstance(code, str):
        return ""

    # Remove string literals (but keep line breaks)
    code = re.sub(r'\".*?\"|\'.*?\'', '', code)

    # Remove symbols but preserve line breaks and indentation
    code = re.sub(r'[^\w\s\n]', '', code)

    code = code[:10000]  # Truncate long files

    # Normalize to lowercase
    return code.lower()

In [ ]:
df["clean_code"] = df["code"].apply(preprocess_code)

In [ ]:
missing = df["clean_code"].isnull().sum()
print(f"Missing code rows: {missing}")

Missing code rows: 0


In [ ]:
df["clean_line_count"] = df["clean_code"].apply(lambda x: len(x.splitlines()))
df["clean_size"] = df["clean_code"].apply(lambda x: len(x.encode("utf-8")))

In [ ]:
df

,id,file_path,file_size,line_count,extension,language,code,clean_code,clean_line_count,clean_size
0,1,Markdown/000001.md,34784,572,md,Markdown,# Contributing\n\n| Component | Bui...,contributing\n\n component build ...,186,10000
1,2,XML/000002.props,3013,44,props,XML,"﻿<Project ToolsVersion=""15.0"" xmlns=""http://sc...",project toolsversion xmlns\n propertygroup\n ...,44,1812
2,3,Text/000003.txt,1076,21,txt,Text,The MIT License (MIT)\n\nCopyright (c) 2015 Mi...,the mit license mit\n\ncopyright c 2015 micros...,21,1026
3,4,Markdown/000004.md,8105,84,md,Markdown,# Azure SDK for .NET\n\n[![Packages](https://i...,azure sdk for net\n\npackageshttpsimgshieldsi...,84,7244
4,5,Markdown/000005.md,2763,41,md,Markdown,<!-- BEGIN MICROSOFT SECURITY.MD V0.0.5 BLOCK ...,begin microsoft securitymd v005 block \n\n se...,41,2523
...,...,...,...,...,...,...,...,...,...,...
86222,86229,SQL/086229.sql,333,18,sql,SQL,CREATE SCHEMA TestSchema;\nGO\n\nCREATE TABLE ...,create schema testschema\ngo\n\ncreate table t...,18,253
86223,86230,SQL/086230.sql,420,8,sql,SQL,"WITH a AS (SELECT * FROM (VALUES(1),(2),(3),(4...",with a as select from values12345678910 as aa...,8,304
86224,86231,SQL/086231.sql,333,18,sql,SQL,CREATE SCHEMA TestSchema;\nGO\n\nCREATE TABLE ...,create schema testschema\ngo\n\ncreate table t...,18,253
86225,86232,SQL/086232.sql,420,8,sql,SQL,"WITH a AS (SELECT * FROM (VALUES(1),(2),(3),(4...",with a as select from values12345678910 as aa...,8,304


In [ ]:
mean_file_size = df["file_size"].mean()
print(f"Mean original file size: {mean_file_size:.2f} bytes")
mean_clean_size = df["clean_size"].mean()
print(f"Mean cleaned code size: {mean_clean_size:.2f} bytes")

Mean original file size: 13460.08 bytes
Mean cleaned code size: 2490.49 bytes


In [ ]:
df[['line_count','language','clean_code']].head()


,line_count,language,clean_code
0,572,Markdown,contributing\n\n component build ...
1,44,XML,project toolsversion xmlns\n propertygroup\n ...
2,21,Text,the mit license mit\n\ncopyright c 2015 micros...
3,84,Markdown,azure sdk for net\n\npackageshttpsimgshieldsi...
4,41,Markdown,begin microsoft securitymd v005 block \n\n se...


In [ ]:
df[['line_count','language','clean_code']].tail()

,line_count,language,clean_code
86222,18,SQL,create schema testschema\ngo\n\ncreate table t...
86223,8,SQL,with a as select from values12345678910 as aa...
86224,18,SQL,create schema testschema\ngo\n\ncreate table t...
86225,8,SQL,with a as select from values12345678910 as aa...
86226,18,SQL,create schema testschema\ngo\n\ncreate table t...


# downloading the cleaned new dataset

In [ ]:
df.to_csv("cleaned_dataset.csv", index=False)
df.to_json("cleaned_dataset.json", orient="records", indent=2)
from google.colab import files
files.download("cleaned_new_dataset.csv")
files.download("cleaned_new_dataset.json")